In [7]:
import requests
import time
import csv

API_KEY = '9e9ab06256174d82a01dc73f02418ca2'  # ← REEMPLÁZALO
BASE_URL = 'https://api.rawg.io/api/games'

def obtener_juegos_desde_1995(paginas_por_año=3):
    juegos_por_año = []

    for año in range(1995, 2025):  # de 1995 a 2024 inclusive
        juegos_este_año = []
        for pagina in range(1, paginas_por_año + 1):
            params = {
                'key': API_KEY,
                'dates': f'{año}-01-01,{año}-12-31',
                'page': pagina,
                'page_size': 40,
                'ordering': '-rating'
            }
            response = requests.get(BASE_URL, params=params)
            if response.status_code != 200:
                print(f"Error {response.status_code} en el año {año}, página {pagina}")
                break

            data = response.json()
            for juego in data.get('results', []):
                rating = juego.get('rating')
                if rating and rating > 0 and juego.get('genres'):
                    juegos_este_año.append({
                        'nombre': juego.get('name'),
                        'fecha_lanzamiento': juego.get('released'),
                        'año': año,
                        'puntuacion_media': rating,
                        'generos': ', '.join([g['slug'] for g in juego['genres']])
                    })

            if not data.get('next'):
                break
            time.sleep(1)  # evitar bloqueo

        if not juegos_este_año:
            # Añadir entrada vacía si no hay datos para el año
            juegos_por_año.append({
                'nombre': 'Sin datos',
                'fecha_lanzamiento': None,
                'año': año,
                'puntuacion_media': None,
                'generos': 'sin_datos'
            })
        else:
            juegos_por_año.extend(juegos_este_año)

    return juegos_por_año

def guardar_csv(datos, archivo='videojuegos_1995_2024_completo.csv'):
    with open(archivo, 'w', newline='', encoding='utf-8') as f:
        campos = ['nombre', 'fecha_lanzamiento', 'año', 'puntuacion_media', 'generos']
        writer = csv.DictWriter(f, fieldnames=campos)
        writer.writeheader()
        writer.writerows(datos)

# Ejecutar
if __name__ == '__main__':
    juegos = obtener_juegos_desde_1995(paginas_por_año=3)
    guardar_csv(juegos)


In [8]:
import pandas as pd
import plotly.express as px

# Cargar los datos
df = pd.read_csv('videojuegos_1995_2024_completo.csv')

# Filtrar juegos con datos válidos
df = df.dropna(subset=['puntuacion_media', 'año', 'generos'])
df['año'] = df['año'].astype(int)

# Tomar solo el primer género de la lista
df['genero_principal'] = df['generos'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else 'sin_datos')

# Agrupar por año y género, y calcular media de puntuación
pivot = df.groupby(['genero_principal', 'año'])['puntuacion_media'].mean().reset_index()

# Convertir a formato matriz para heatmap
heatmap_data = pivot.pivot(index='genero_principal', columns='año', values='puntuacion_media')

# Crear heatmap
fig = px.imshow(
    heatmap_data,
    labels=dict(x="Año", y="Género", color="Puntuación media"),
    title="🔥 Mapa de calor de la puntuación media por género y año (1995–2024)",
    text_auto=".2f",
    aspect="auto",
    color_continuous_scale="Viridis"
)

fig.update_layout(template="plotly_dark")
fig.show()
